In [1]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_community.graphs import Neo4jGraph
# from neo4j_graphrag import Text12Graph
from langchain_neo4j import Neo4jVector
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_neo4j import Neo4jGraph
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from langchain_neo4j import GraphCypherQAChain
# from langchain.graphs.neo4j_graph import Neo4jGraph
from dotenv import load_dotenv

In [2]:
from dotenv import load_dotenv

In [3]:
os.getenv("NGROK_URL")

'https://dereistic-ta-unrobustly.ngrok-free.dev'

In [4]:
template = """Question: {question}

Answer: Let's think step by step."""

In [5]:
prompt = ChatPromptTemplate.from_template(template)

In [6]:
model = OllamaLLM(model="gpt-oss:20b-cloud")

chain = prompt | model

In [7]:
# from neo4j import GraphDatabase
graph = Neo4jGraph(
    url=os.getenv("URL"),
    username=os.getenv("USERNAME"),
    password=os.getenv("PASSWORD")
)

In [28]:
chain_cypher = GraphCypherQAChain.from_llm(llm=model, graph=graph, verbose=True,allow_dangerous_requests = True)

## Not Cool

In [12]:
import ollama

In [13]:
prompt = f"Can you read anything from this {chain_cypher}"

In [ ]:
response = ollama.generate(model='gpt-oss:20b', prompt=prompt)

## USing the colab version of ollama

In [22]:
from ollama import Client

client = Client(host=os.getenv("NGROK_URL"))

response = client.chat(
    model="ministral-3:14b",
    messages=[{"role": "user", "content": "Hello"}]
)

print(response["message"]["content"])

ResponseError: The endpoint dereistic-ta-unrobustly.ngrok-free.dev is offline.

ERR_NGROK_3200
 (status code: 404)

## Asking questions based on the knowledge graph.

In [50]:
question = "In which comopany is Aum an intern?"
result = chain_cypher.invoke({"query": question})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Aum'})-[:WORKS_AT {role: 'intern'}]->(c:Company)
RETURN c.name AS Company;
Full Context:
[]

> Finished chain.


In [51]:
result

{'query': 'In which comopany is Aum an intern?',
 'result': 'I don’t know the answer.'}

In [11]:
result['result']

'Soumil Binani.'

## Inserting new memories :

In [ ]:
new_memory = "Shrey Baldev is the founder of Moon Finance"
documents = [Document(page_content=new_memory)]

In [23]:
graph_transformer = LLMGraphTransformer(llm=model)
# graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)
# graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [24]:
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [25]:
graph_documents[0].nodes  # list of nodes
graph_documents[0].relationships  # list of relationships

[Relationship(source=Node(id='Shrey Baldev', type='Person', properties={}), target=Node(id='Moon Finance', type='Company', properties={}), type='FOUNDED', properties={})]

In [26]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Moon Finance', type='Company', properties={}), Node(id='Shrey Baldev', type='Person', properties={})], relationships=[Relationship(source=Node(id='Shrey Baldev', type='Person', properties={}), target=Node(id='Moon Finance', type='Company', properties={}), type='FOUNDED', properties={})], source=Document(metadata={}, page_content='Shrey Baldev is the founder of Moon Finance'))

In [27]:
await update_graph_in_neo4j(graph_documents)

NameError: name 'update_graph_in_neo4j' is not defined

In [28]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Shrey Baldev', type='Person', properties={}), Node(id='Moon Finance', type='Company', properties={})]
Relationships:[Relationship(source=Node(id='Shrey Baldev', type='Person', properties={}), target=Node(id='Moon Finance', type='Company', properties={}), type='FOUNDERS', properties={})]


In [ ]:
# Aiyo KG hatt naa jaye puraka pura:
summary = driver.execute_query(cypher_query_manual, database_="neo4j").summary
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

# Isnertion of new mem, new approach (via API calling)

In [43]:
user_prompt = "Aum was an intern at moon finance"

In [44]:
from generate_cypher_query import generate_response

In [45]:
cypher_query = generate_response(user_prompt)
# graph.add_graph_documents([user_prompt]) # THis should only be used when query is in structured format.
#TODO: Also check the text2cypher model on hf

In [46]:
graph.query(cypher_query)

[]

In [47]:
cypher_query

'MERGE (p:Person {name:"Aum"}) \nMERGE (c:Company {name:"Moon Finance"}) \nMERGE (p)-[:INTERN_AT]->(c);'